# Routing Topology: OSPF using FRRouting with Chameleon

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


## Configure the Environment


In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fablib_local"))
if module_path not in sys.path:
    sys.path.append(module_path)

from chameleon_utils.chameleon_config import *

load_chameleon_rc_environment(chameleon_rc_file=f"{os.environ['HOME']}/work/fablib_local_private_config/Chameleon-openrc.sh")

from my_experiment.frrouting_experiment import FRRouting_Experiment


## Create the Experiment


In [ ]:
frr_experiment = FRRouting_Experiment('FRRouting_OSPF')

In [ ]:
#frr_experiment.load()
#frr_experiment.display()

## (Optional): Query for Available Tesbed Resources and Settings

In [ ]:
try:
    frr_experiment.list_resources()

except Exception as e:
    print(f"Exception: {e}")

## Define Experiment

The following creates private layer 2 networks on three sites including a OSPF gateway routers that propogate routes acrross the topology. 


In [ ]:
frr_sites = { 'dall': { 'site': 'DALL', 'facility': 'FABRIC', 'node_count': 2 },
              'salt': { 'site': 'SALT', 'facility': 'FABRIC', 'node_count': 2 },
              'chameleon': { 'site': 'STAR', 'facility': 'CHI@UC', 'node_count': 2 },
              'utah': { 'site': 'UTAH', 'facility': 'FABRIC', 'node_count': 2 }
            }
    
frr_links = { 'Link12': ('dall','salt'),
              'Link23': ('salt','chameleon'),
              'Link31': ('chameleon','dall'),
              'Link24': ('utah','salt')
            }

In [ ]:
frr_experiment.configure(frr_sites, frr_links)

In [ ]:
frr_experiment.deploy()

## Run the GUI and Interact with the Experiment

In [ ]:
#frr_experiment.load()
frr_experiment.display()

## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    frr_experiment.fablib.delete_all()
    #slice.delete()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
frr_sites = { 'dall': { 'site': 'DALL', 'facility': 'FABRIC', 'node_count': 2 },
              'utah': { 'site': 'UTAH', 'facility': 'FABRIC', 'node_count': 2 },

              'salt': { 'site': 'SALT', 'facility': 'FABRIC', 'node_count': 2 },
              'wash': { 'site': 'WASH', 'facility': 'FABRIC', 'node_count': 2 },
              'tacc': { 'site': 'TACC', 'facility': 'FABRIC', 'node_count': 2 },
              'max':  { 'site': 'MAX', 'facility': 'FABRIC', 'node_count': 2 },
              'ncsa': { 'site': 'NCSA', 'facility': 'FABRIC', 'node_count': 2 },
              'cham': { 'site': 'STAR', 'facility': 'CHI@UC', 'node_count': 2 },
            }
    
frr_links = { 'Link_dall_tacc': ('dall','tacc'),
              'Link_cham_ncsa': ('cham','ncsa'),
              'Link_salt_utah': ('salt','utah'),
              'Link_wash_max':  ('wash','max'),
              'Link_utah_dall': ('utah','dall'),
              'Link_dall_wash': ('dall','wash'),
              'Link_wash_cham': ('wash','cham'),
              'Link_cham_utah': ('cham','utah'),
              'Link_cham_dall': ('cham','dall'), 
            }

In [ ]:
frr_experiment.wait_jupyter(timeout=20000)

In [ ]:
frr_experiment.execute_on_all_edge_nodes(f'rm -rf fabric_node_tools')
print('deploy3')
frr_experiment.upload_directory_to_all_edge_nodes('fabric_node_tools','.')
print('deploy4')
frr_experiment.execute_on_all_edge_nodes(f'chmod +x fabric_node_tools/*.sh')
print('deploy5')
frr_experiment.configure_devs()
print('deploy6')
frr_experiment.configure_routers()
print('deploy7')
command = f'sudo yum install -y -q iperf3 iproute-tc && sudo ./fabric_node_tools/host_tune_redhat.sh'
frr_experiment.execute_on_all_edge_nodes(f'{command}')
print('deploy8')